In [ ]:
import torch
from transformers import AutoTokenizer
import json
import requests
from transformers import LayoutLMModel
from pytorch_lightning import LightningModule
import torch.nn as nn
from infer.model.loss import BboxLoss
from infer.model.spade_model import RelationTagger
import json

jsonfile =  json.load(open('check.jsonl')) 

tokenizer = AutoTokenizer.from_pretrained("microsoft/layoutlm-base-uncased")
def get_bbox(jsonl_file):
    w = jsonl_file['img_sz']['width']
    h = jsonl_file['img_sz']['height']
    bboxes = [[ int(x[0][0]*1000/w),int(x[0][1]*1000/h) ,int(x[2][0]*1000/w),int(x[2][1]*1000/h)] for x in jsonl_file['coord']]
    return  torch.tensor(bboxes)

In [ ]:
from torch import Tensor


words = jsonfile['text']
normalized_word_boxes = get_bbox(jsonfile)
encoding = tokenizer(" ".join(words), return_tensors='pt')
token_boxes = []
i = 0
maps = []
maps.extend([i])
result = {}
for word, box in zip(words, normalized_word_boxes):
    word_tokens = tokenizer.tokenize(word)
    token_boxes.extend([box] * len(word_tokens))
    i += 1
    maps.extend([i]*len(word_tokens))

maps.extend([i+1])

token_boxes = [[0, 0, 0, 0]] + token_boxes + [[1000, 1000, 1000, 1000]]

input_ids = encoding["input_ids"]
attention_mask = encoding["attention_mask"]
token_type_ids = encoding["token_type_ids"]
bbox = torch.tensor([token_boxes])
maps = torch.tensor(maps).unsqueeze(0)

In [ ]:
# m = np.array(maps.cpu())
# m = m.tolist()
# box_index =  np.unique(m).tolist()
# [[item]*m.count(item) for item in box_index]
# input_ids.shape
model = LayoutLMModel.from_pretrained(
            "microsoft/layoutlm-base-uncased", local_files_only=True)

In [ ]:
outputs = model(input_ids=input_ids,bbox=bbox,attention_mask=attention_mask,token_type_ids=token_type_ids)
outputs.last_hidden_state.shape

reduce_size = 256

ln = nn.Linear(model.config.hidden_size, reduce_size)

last_hidden_state = ln(outputs.last_hidden_state)
last_hidden_state.shape

zeros_temp = torch.zeros(reduce_size)

def tranfer_maps(maps: Tensor):
    m = []
    maps = torch.squeeze(maps, 0)
    uniq, count= maps.unique(return_counts=True)
    for i in range(len(count)):
        m.append(torch.stack([uniq[i]]*count[i]))
    return m

def reduce_shape(last_hidden_state, maps):
    i = 0
    device = last_hidden_state.device
    reduce = []
    for j  in range(len(maps)):
        g_token = maps[j]
        ten = zeros_temp.to(device)
        for k in range(g_token.shape[0]):
            ten += last_hidden_state[0][i]
            i += 1
        ten = ten/g_token.shape[0]
        reduce.append(ten)
    reduce = torch.stack(reduce).to(device)
    return reduce



reduce = reduce_shape(last_hidden_state, tranfer_maps(maps))
# # maps

torch.mean(reduce -reduce)
# tranfer_maps(maps)
# maps = 
# reduce_shape(last_hidden_state, tranfer_maps(maps) )

In [ ]:

# %reload_ext autoreload
from models.layout_parsing import LitBaseParsing
%load_ext autoreload
%autoreload 2
from models.base import  LitLayoutParsing
model = LitBaseParsing()

In [ ]:
# totalGraph =  model(input_ids, attention_mask, token_type_ids, bbox,maps)

# totalGraph.shape
dummy_input = torch.zeros(3,126)
dummy_input[1]

In [ ]:
# dummy_input = torch.zeros_like(input_ids).cuda()
# dummy_boxes = torch.zeros_like(bbox).cuda()
# dummy_maps = torch.zeros_like(maps).cuda()
import torch
dummy_input = torch.zeros(1,196).cuda()
dummy_boxes = torch.zeros(1,196,4).cuda()
dummy_maps = torch.zeros(1,196).cuda()
# totalGraph[0].shape
# S = totalGraph[0].unsqueeze(0)
# G = totalGraph[1].unsqueeze(0)
# dummy_maps.shape
# dummy_maps.unsqueeze(0).shape

In [1]:
import onnxruntime as ort
import numpy as np


providers = [
            # 'TensorrtExecutionProvider',
                     'CUDAExecutionProvider',
                     'CPUExecutionProvider']

ort_sess = ort.InferenceSession(
                                path_or_bytes = 'resources/onnxLitLP_03-03-2023_15-45-29_fp32.onnx'
                                , providers= providers)

In [6]:
input_name = ort_sess.get_inputs()
# output_name = ort_sess.get_outputs()[0].name
input_name[1].name


# outputs = ort_sess.run(None, 
#                         {
#                             'input_1': input_ids.numpy()
#                             'input_2': input_ids.numpy()
#                             'input_3': input_ids.numpy()
#                             'input_4': input_ids.numpy()
#                             'input_5': input_ids.numpy()
#                         }
#                         )


'input_2'